In [1]:
from PYME.LMVis import VisGUI

%gui wx

Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.io
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.io
Trying to load 3rd party recipe module pyme_picasso.recipe_modules.conversion
Loaded 3rd party recipe module pyme_picasso.recipe_modules.conversion
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.localisations
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.localisations
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Trying to load 3rd party recipe module pyme_4pi.recipe_modules.conversion
Loaded 3rd

In [2]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations
New Canvas


DEBUG:PYME.LMVis.visCore:Creating VisGUI menu bar
DEBUG:PYME.LMVis.Extras:Initializing QPobjectSegment plugin
DEBUG:PYME.LMVis.Extras:Initializing animation plugin
DEBUG:PYME.LMVis.Extras:Initializing annotation plugin
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:62: wxPyDeprecationWarning: Call to deprecated item. Use SetToolTip instead.
  self.bAdjustSnake.SetToolTipString('Adjust the parameters of fo the "snake" (active contour) used for curve locking')
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:73: wxPyDeprecationWarning: Call to deprecated item. Use InsertItem instead.
  self.lLabels.InsertStringItem(i, '%d' % i)
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:74: wxPyDeprecationWarning: Call to deprecated item. Use SetItem instead.
  self.lLabels.SetStringItem(i, 1, 'Structure %d' % i)
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:76: wxPyDeprecationWarning: Call to deprecate

Saved Views are deprecated -  use the ones with animation instead and remove me!
Creating fold panel
GenPanels
Creating datasource panel
Image list size: 1
recipe.execute()
0
OpenGL - Version: b'4.6.0 NVIDIA 516.94'
Shader - Version: b'4.60 NVIDIA'
GL.GL_MAX_SAMPLES: 32, GL.GL_SAMPLES: 4


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.DefaultShaderProgram'>


Ev Idle
('points', 't')
refreshed
Input datasources:  []
Creating layers GUI
height:  0
NB best size: wx.Size(0, 0)


In [61]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from PYME.Analysis.points import cluster_morphology

#Parameters to change
savedir = 'K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111\\analysis'
c_info = "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111\\count_info_V1.npy"
save_loc = "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111"
data = save_loc + '\\cluster_output_V1.npy'
circ = save_loc + '\\circular_sims_V1.npy'
line = save_loc + '\\linear_sims_V1.npy'
ver = 'V1'
min_size = 10      # currently not used
max_size = 50     # currently not used
m_dist = 3.75 # distance between monomers in nanometers
lpm = 3.23 # factor that changes # of localizations per cluster distribution into one that looks like
            # the number of molecules per cluster in the counting data
lp = 5.65 # this is the average localization precision from the three Rtn4 experiments
# Import data
real = np.load(data, allow_pickle=True)
#real = real.item()
circ_sim = np.load(circ, allow_pickle=True)
#circ_sim = circ_sim.item()
line_sim = np.load(line, allow_pickle=True)
#line_sim = line_sim.item()
count_info = np.load(c_info, allow_pickle=True)
count_info = count_info.item()

In [69]:
min_oli = int(np.rint(min_size/lpm))
max_oli = int(np.rint(max_size/lpm))
frequency = count_info['frequency'][min_oli:max_oli]
oli_size = count_info['oli-size'][min_oli:max_oli]

array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.])

In [59]:
clusters = []
rng = np.random.default_rng()

frequency 
for a in range(len(oli_size)):                                  # for each oligomer size
    for b in range(int(frequency[a])):                          # for the number of times that oligomer size occurs
        pois = rng.poisson(lpm,int(oli_size[a]))                     # determine number of localizations each molecule has
        total = sum(pois)                                       # find total number of localizations for this cluster
        count = 0
        points = np.zeros((total,3))
        for c in range(int(oli_size[a])):                            # for each of molecule in this cluster
            for d in range(pois[c]):                            # for each localization in each molecule in this cluster
                points[count] = np.array([m_dist*c,0,0])
                points[count][0] = points[count][0] + rng.normal(loc=0,scale=lp,size=1)
                points[count][1] = points[count][1] + rng.normal(loc=0,scale=lp,size=1)
                points[count][2] = points[count][2] + rng.normal(loc=0,scale=lp,size=1)
                count = count + 1
        clusters.append(points)

# Unpack wanted measurements
for z in range(len(real)):
    if z == 0:
        real_ani = real[z]['anisotropy']
        circ_ani = circ_sim[z]['anisotropy']
        line_ani = line_sim[z]['anisotropy']
        ccounts = real[z]['count']
    else:
        np.concatenate((real_ani, real[z]['anisotropy']))
        np.concatenate((circ_ani, circ_sim[z]['anisotropy']))
        np.concatenate((line_ani, line_sim[z]['anisotropy']))
        np.concatenate((ccounts, real[z]['count']))

min_filt = ccounts > min_size
max_filt = ccounts < max_size
size_filt = min_filt * max_filt
real_ani_filt = real_ani[size_filt]
circ_ani_filt = circ_ani[size_filt]
line_ani_filt = line_ani[size_filt]
min_csize = int(np.rint(min_size/lpm))
max_csize = int(np.rint(max_size/lpm))
# Round frequency to nearest integer
frequency = np.rint(count_info['frequency'])[min_csize:max_csize]
# Grab oligomer size bins
oli_size = count_info['oli-size'][min_csize:max_csize]

cluster_anisotropy = np.zeros(len(clusters))
for e in range(len(clusters)):
    if len(clusters[e]) > 2:
        cluster_meas = cluster_morphology.measure_3d(x=clusters[e][:,0],y=clusters[e][:,1],
                                                  z=clusters[e][:,2])
        cluster_anisotropy[e] = cluster_meas['anisotropy'][0]
    else:
        continue

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

fig = plt.figure(figsize=[3,2])
plt.hist(circ_ani_filt,bins=20, range=[0,1], alpha=0.3,label='Circular', 
         density=True, color='gold', edgecolor='k')
plt.hist(line_ani_filt,bins=20, range=[0,1], alpha=0.3,label='Linear', 
         density=True, color='lime', edgecolor='k')
plt.hist(real_ani_filt,bins=20, range=[0,1], alpha=0.3,label='Real', 
         density=True, color=CB_color_cycle[5], edgecolor='k')
plt.legend(loc='upper right', bbox_to_anchor=(1.02, 1.03),frameon=False, markerfirst=False)
plt.xlabel('Anisotropy')
plt.ylabel('Normalized frequency')
plt.show()
ani_save = savedir + '\\Cluster_anisotropy_' + ver + '.png'
plt.savefig(ani_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0, metadata=None)

fig = plt.figure(figsize=[3,2])
plt.hist(cluster_anisotropy, bins=20, range=[0,1], alpha=0.3,label='Sim-mono', 
         density=True, color=CB_color_cycle[6], edgecolor='k')
plt.hist(real_ani_filt,bins=20, range=[0,1], alpha=0.3,label='Real', 
         density=True, color=CB_color_cycle[5], edgecolor='k')
plt.legend(loc='upper right', bbox_to_anchor=(1.02, 1.03),frameon=False, markerfirst=False)
plt.xlabel('Anisotropy')
plt.ylabel('Normalized frequency')
plt.show()
mono_sim_save = savedir + '\\Monomer_sim_anisotropy_' + ver + '-' + str(m_dist) + '.png'
plt.savefig(mono_sim_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0, metadata=None)